This notebook is a supplement to the paper ["The Orthologic of Epistemic Modals"](https://escholarship.org/uc/item/0ss5z8g3) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu) and [Matthew Mandelkern](mandelkern@nyu.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations. For example, we treat the logical principle $\Box\varphi \vdash \varphi$, corresponding to the lattice inequality $\Box a\leq a$, as the equation $\Box a = \Box a\wedge a$.

## Outline

**1. [Lattice axioms](#1)**

**2. [Bounded lattice axioms](#2)**

**3. [Ortholattice axioms](#3)**

**4. [Boolean subalgebra axioms](#4)**

**5. [Modal axioms](#5)**

**6. [Epistemic axioms](#6)**

**7. [Conditional axioms](#7)**

**8. [Independence of axioms](#8)**

**9. [Example proofs](#9)**

**10. [Avoiding collapse](#10)**

**11. [Modalized Import-Export](#11)**

**12. [Qualified Collapse](#12)**

**13. [Provable principles for which Prover9 does not find a proof](#13)**

In [1]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## 1. Lattice axioms<a id='1'></a>

In [2]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

## 2. Bounded lattice axioms<a id='2'></a>

In [3]:
bot = read_expr('And(x,Bot) = Bot')
top = read_expr('Or(x,Top) = Top')

bounded_lattice = lattice + [bot,top]

## 3. Ortholattice axioms<a id='3'></a>

In [4]:
contra = read_expr('And(x,Not(x)) = Bot')
lem = read_expr('Or(x,Not(x)) = Top')

invol = read_expr('Not(Not(x)) = x')

de_morgan = read_expr('Not(And(x,y)) = Or(Not(x),Not(y))')

ortho_lattice = bounded_lattice + [contra,lem,invol,de_morgan]

In [5]:
#We do not want distributivity, but we consider it below
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

In [6]:
#Check that the De Morgan law dual to de_morgan above follows
goal = read_expr('Not(Or(x,y)) = And(Not(x),Not(y))')
prover = Prover9Command(goal, assumptions = ortho_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27464 was started by halcrow on iMac.local,
Wed Feb  9 20:16:58 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 70.


1 Not(Or(x,y)) = And(Not(x),Not(y)).  [goal].
16 Not(Not(x)) = x.  [assumption].
17 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
18 Not(Or(c1,c2)) != And(Not(c1),Not(c2)).  [deny(1)].
49 Not(Or(Not(x),Not(y))) = And(x,y).  [para(17(a,1),16(a,1,1))].
168 Not(Or(x,Not(y))) = And(Not(x),y).  [para(16(a,1),49(a,1,1,1))].
277 Not(Or(x,y)) = And(Not(

In [7]:
#Check that the distributive law does not follow from ortholattice axioms
goal = dist
mb = MaceCommand(goal, assumptions = ortho_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

c3 = 4.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 0.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(2,4) = 1.
Or(2,5) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.
Or(3,4) = 1.
Or(3,5) = 1.
Or(4,0)

In [8]:
#Check that adding the pseudocomplementation principle allows the derivability of distributivity
pseudo = read_expr('And(x,y) = Bot -> y = And(y,Not(x))')
goal = dist
prover = Prover9Command(goal, assumptions = ortho_lattice + [pseudo])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27468 was started by halcrow on iMac.local,
Wed Feb  9 20:16:58 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.43 (+ 0.02) seconds.
% Length of proof is 94.
% Level of proof is 20.
% Maximum clause weight is 22.000.
% Given clauses 309.


1 And(x,y) = Bot -> y = And(y,Not(x)).  [assumption].
2 And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [goal].
3 Or(x,x) = x.  [assumption].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
7 Or(x,Or(y,z)) = Or(Or(x,y),z).  [assumption].
8 Or(Or(x,y),z) = Or(x,Or(y,z)).  [copy(7),flip(a)].
9 And(x,And(y,z)) = And(And(x,y),z).  

In [9]:
#Conversely, adding distributivity allows the derivation of the pseudocomplementation principle
goal = pseudo
prover = Prover9Command(goal, assumptions = ortho_lattice + [dist])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27470 was started by halcrow on iMac.local,
Wed Feb  9 20:16:58 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 25.
% Level of proof is 9.
% Maximum clause weight is 13.000.
% Given clauses 50.


1 And(x,y) = Bot -> y = And(y,Not(x)).  [goal].
4 Or(x,y) = Or(y,x).  [assumption].
5 And(x,y) = And(y,x).  [assumption].
10 Or(x,And(x,y)) = x.  [assumption].
11 And(x,Or(x,y)) = x.  [assumption].
12 And(x,Bot) = Bot.  [assumption].
13 Or(x,Top) = Top.  [assumption].
15 Or(x,Not(x)) = Top.  [assumption].
16 Not(Not(x)) = x.  [assumption].
17 Not(A

## 4. Boolean subalgebra axioms<a id='4'></a>

In [10]:
B_bot = read_expr('B(Bot)')
B_not = read_expr('B(x) -> B(Not(x))')
B_and = read_expr('(B(x) & B(y)) -> B(And(x,y))')
B_or = read_expr('(B(x) & B(y)) -> B(Or(x,y))')
B_dist = read_expr('((B(x) & B(y)) & B(z)) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

ortho_boolean_lattice = ortho_lattice + [B_bot, B_not, B_and, B_or, B_dist]

In [11]:
#Check that the other distributive law dual to B_dist above follows
goal = read_expr('((B(x) & B(y)) & B(z)) -> Or(x,And(y,z)) = And(Or(x,y),Or(x,z))')
prover = Prover9Command(goal, assumptions = ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27472 was started by halcrow on iMac.local,
Wed Feb  9 20:16:58 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.07 (+ 0.01) seconds.
% Length of proof is 63.
% Level of proof is 10.
% Maximum clause weight is 19.000.
% Given clauses 52.


1 B(x) -> B(Not(x)).  [assumption].
2 B(x) & B(y) -> B(And(x,y)).  [assumption].
3 B(x) & B(y) -> B(Or(x,y)).  [assumption].
4 B(x) & B(y) & B(z) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z)).  [assumption].
5 B(x) & B(y) & B(z) -> Or(x,And(y,z)) = And(Or(x,y),Or(x,z)).  [goal].
6 Or(x,x) = x.  [assumption].
7 And(x,x) = x.  [assumption].
8 Or(x,y) = 

## 5. Modal axioms<a id='5'></a>

In [12]:
box_and = read_expr('Box(And(x,y)) = And(Box(x),Box(y))')
box_top = read_expr('Box(Top) = Top')
diamond_dual = read_expr("Diamond(x) = Not(Box(Not(x)))")

modal_ortho_boolean_lattice = ortho_boolean_lattice + [box_and, box_top, diamond_dual]

In [13]:
#Check that Diamond distributes over disjunction
goal = read_expr('Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y))')
prover = Prover9Command(goal, assumptions = modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27474 was started by halcrow on iMac.local,
Wed Feb  9 20:16:59 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.02 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 16.000.
% Given clauses 107.


5 Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y)).  [goal].
20 Not(Not(x)) = x.  [assumption].
21 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
28 Box(And(x,y)) = And(Box(x),Box(y)).  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
31 Diamond(Or(c1,c2)) != Or(Diamond(c1),Diamond(c2)).  [deny(5)].
32 Not(Box(Not(Or(c1,c2)))) != Or(N

## 6. Epistemic axioms<a id='6'></a>

In [14]:
factive = read_expr('Box(x) = And(Box(x),x)')
episteme = read_expr('And(x,Diamond(Not(x))) = Bot')

epistemic_ortho_boolean_lattice = modal_ortho_boolean_lattice + [factive, episteme]

In [15]:
#Check that another form of Wittgenstein sentence is contradictory
goal = read_expr('And(Not(x),Diamond(x)) = Bot')
prover = Prover9Command(goal, assumptions = epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

#Check that "p or it must be that not p" is derivable
goal = read_expr('Or(x,Box(Not(x))) = Top')
prover = Prover9Command(goal, assumptions = epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27476 was started by halcrow on iMac.local,
Wed Feb  9 20:16:59 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 0.


5 And(Not(x),Diamond(x)) = Bot.  [goal].
20 Not(Not(x)) = x.  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
33 And(x,Diamond(Not(x))) = Bot.  [assumption].
34 And(x,Not(Box(x))) = Bot.  [copy(33),rewrite([30(2),20(2)])].
35 And(Not(c1),Diamond(c1)) != Bot.  [deny(5)].
36 $F.  [copy(35),rewrite([30(4),34(7)]),xx(a)].

=================

In [16]:
#Check that Diamond p does not entail p
goal = read_expr('Diamond(x) = And(Diamond(x),x)')
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 5.
Box(4) = 2.
Box(5) = 5.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 2.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 5.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 2.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 5.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2

In [17]:
#Although "p and might not p" is inconsistent, "might p and must might not p" is consistent
goal = read_expr('And(Diamond(x),Box(Diamond(Not(x)))) = Bot')
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 10

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 7.
Box(3) = 5.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 9.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 6.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 8.
Diamond(9) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 2.
And(2,5) = 0.
And(2,6) = 0.
And(2,7) = 7.
And(2,8) = 0.
And(2,9) = 7.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 5.
And(3,6

## 7. Conditional axioms<a id='7'></a>

In [18]:
if_and = read_expr('If(x,And(y,z)) = And(If(x,y),If(x,z))')
if_top = read_expr('If(x,Top) = Top')

ident = read_expr('If(x,x) = Top') 

simp_mp = read_expr('B(y) -> And(If(x,y),x) = And(And(If(x,y),x),y)')
simp_cs = read_expr('B(y) -> And(x,y) = And(And(x,y),If(x,y))')
simp_mt = read_expr('B(y) -> And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))')

stm = read_expr('If(x,y) = And(If(x,y),Or(Not(x),y))')

simp_caut_trans = read_expr('B(z) -> And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))')
simp_caut_mon = read_expr('B(z) -> And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))')

flat = read_expr('If(x,If(And(x,y),z)) = If(And(x,y),z)')

simp_cem_plus = read_expr('B(x) -> If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))')
simp_ni = read_expr('B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))')

weak_boethius = read_expr('And(Diamond(x),If(x,y)) = And(And(Diamond(x),If(x,y)),Not(If(x,Not(y))))')

scopeless = read_expr('Box(If(x,y)) = If(x,Box(y))')
mod_trans_mon = read_expr('And(If(x,Box(y)),If(And(x,y),z)) = And(If(x,Box(y)),If(x,z))')
must_preserve = read_expr('And(Diamond(And(x,y)),Box(y)) = And(And(Diamond(And(x,y)),Box(y)),If(x,Box(y)))')


cond_ax = [if_and, if_top, ident, 
           simp_mp, simp_cs, simp_mt, 
           stm, simp_caut_trans, simp_caut_mon, 
           flat, simp_cem_plus, simp_ni,
           weak_boethius, scopeless, mod_trans_mon, must_preserve]

cond_modal_ortho_boolean_lattice = modal_ortho_boolean_lattice + cond_ax
cond_epistemic_ortho_boolean_lattice = epistemic_ortho_boolean_lattice + cond_ax

In [19]:
#Check that "If Diamond p, then p" is not valid
goal = read_expr('If(Diamond(x),x) = Top')
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 5.
Box(4) = 4.
Box(5) = 5.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 2.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 5.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 2.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 5.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 5.
If(2,0) = 0

## 8. Independence of Axioms<a id='8'></a>

Not all of the axioms in cond_ax are independent. 

In [20]:
#if_top is derivable from other axioms
goal = if_top
prover = Prover9Command(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice if not (ax == if_top)])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27486 was started by halcrow on iMac.local,
Wed Feb  9 20:16:59 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 6.
% Level of proof is 2.
% Maximum clause weight is 13.000.
% Given clauses 33.


12 If(x,Top) = Top.  [goal].
43 If(x,x) = Top.  [assumption].
56 If(x,If(And(x,y),z)) = If(And(x,y),z).  [assumption].
67 If(c1,Top) != Top.  [deny(12)].
140 If(x,Top) = Top.  [para(43(a,1),56(a,1,2)),rewrite([43(5)])].
141 $F.  [resolve(140,a,67,a)].

============================== end of proof ==========================


In [21]:
#simp_mp and simp_cs are derivable from other axioms

In [22]:
goal = simp_mp
prover = Prover9Command(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice if not (ax == simp_cs or ax == simp_mp)])
prover.prove()
print(prover.proof())

goal = simp_cs
prover = Prover9Command(goal, assumptions = [ax for ax in cond_epistemic_ortho_boolean_lattice if not (ax == simp_cs or ax == simp_mp)])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27488 was started by halcrow on iMac.local,
Wed Feb  9 20:16:59 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 4.23 (+ 0.13) seconds.
% Length of proof is 123.
% Level of proof is 17.
% Maximum clause weight is 27.000.
% Given clauses 1414.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),Not(x)) = And(And(If(y,x),Not(x)),Not(y)).  [assumption].
7 B(x) -> And(If(y,z),If(y,x)) = And(And(If(y,z),If(y,x)),If(And(y,z),x)).  [assumption].
9 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
10 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [goal].
11 Or(x,

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27490 was started by halcrow on iMac.local,
Wed Feb  9 20:17:03 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 1.24 (+ 0.04) seconds.
% Length of proof is 55.
% Level of proof is 10.
% Maximum clause weight is 24.000.
% Given clauses 781.


1 B(x) -> B(Not(x)).  [assumption].
7 B(x) -> And(If(y,z),If(y,x)) = And(And(If(y,z),If(y,x)),If(And(y,z),x)).  [assumption].
9 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
10 B(x) -> And(y,x) = And(And(y,x),If(y,x)).  [goal].
13 Or(x,y) = Or(y,x).  [assumption].
14 And(x,y) = And(y,x).  [assumption].
17 And(x,And(y,z)) =

Mace4 and Prover9 are unable to settle the independence or non-independent of simp_caut_trans, simp_caut_mon, simp_cem_plus, and weak_boethius. However, Mace4 shows that all other axioms (excluding if_top, simp_mp, and simp_cs) are independent of the rest.

In [23]:
for ax in cond_ax:
    if not (ax == if_top or ax == simp_mp or ax == simp_cs or ax == simp_caut_trans or ax == simp_caut_mon or ax == simp_cem_plus or ax == weak_boethius):
        print(f"Is {ax} independent of the other axioms?")
        mace = Mace()
        print(mace.build_model(ax, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice if not axiom == ax]))
        print("\n")

Is (If(x,And(y,z)) = And(If(x,y),If(x,z))) independent of the other axioms?
True


Is (If(x,x) = Top) independent of the other axioms?
True


Is (B(y) -> (And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x)))) independent of the other axioms?
True


Is (If(x,y) = And(If(x,y),Or(Not(x),y))) independent of the other axioms?
True


Is (If(x,If(And(x,y),z)) = If(And(x,y),z)) independent of the other axioms?
True


Is (B(x) -> (Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))))) independent of the other axioms?
True


Is (Box(If(x,y)) = If(x,Box(y))) independent of the other axioms?
True


Is (And(If(x,Box(y)),If(And(x,y),z)) = And(If(x,Box(y)),If(x,z))) independent of the other axioms?
True


Is (And(Diamond(And(x,y)),Box(y)) = And(And(Diamond(And(x,y)),Box(y)),If(x,Box(y)))) independent of the other axioms?
True




Next we verify that the non-simple versions of the simple conditional axioms are not derivable.

In [56]:
mp = read_expr('And(If(x,y),x) = And(And(If(x,y),x),y)')
cs = read_expr('And(x,y) = And(And(x,y),If(x,y))')
mt = read_expr('And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))')
caut_trans = read_expr('And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))')
caut_mon = read_expr('And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))')
cem_plus = read_expr('If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))')
ni = read_expr('Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))')

non_simp_ax = [mp, cs, mt, caut_trans, caut_mon, cem_plus,ni]

In [57]:
for ax in non_simp_ax:
    print(f"Is {ax} a non-theorem?")
    mace = Mace()
    print(mace.build_model(ax, assumptions = cond_epistemic_ortho_boolean_lattice))
    print("\n")

Is (And(If(x,y),x) = And(And(If(x,y),x),y)) a non-theorem?
True


Is (And(x,y) = And(And(x,y),If(x,y))) a non-theorem?
True


Is (And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))) a non-theorem?
True


Is (And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))) a non-theorem?
True


Is (And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))) a non-theorem?
True


Is (If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))) a non-theorem?
True


Is (Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))) a non-theorem?
True




## 9. Example Proofs<a id='9'></a>

Next we show that the "K" principle holds for our $\Box$ and $\to$: $\Box\varphi\wedge\Box(\varphi\to\psi)\vdash \Box\psi$.

In [24]:
goal = read_expr('And(Box(x),Box(If(x,y))) = And(And(Box(x),Box(If(x,y))),Box(y))')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27503 was started by halcrow on iMac.local,
Wed Feb  9 20:17:23 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.10 (+ 0.01) seconds.
% Length of proof is 60.
% Level of proof is 10.
% Maximum clause weight is 24.000.
% Given clauses 267.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
11 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
12 And(Box(x),Box(If(x,y))) = And(And(Box(x),Box(If(x,y))),Box(y)).  [goal].
14 And(x,x) = x.  [assumption].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [as

Next we show $\Box\varphi\wedge (\varphi\to\psi)\vdash\psi$

In [25]:
goal = read_expr('And(Box(x),If(x,y)) = And(And(Box(x),If(x,y)),y)')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27505 was started by halcrow on iMac.local,
Wed Feb  9 20:17:23 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 1.15 (+ 0.04) seconds.
% Length of proof is 57.
% Level of proof is 10.
% Maximum clause weight is 24.000.
% Given clauses 724.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
11 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
12 And(Box(x),If(x,y)) = And(And(Box(x),If(x,y)),y).  [goal].
14 And(x,x) = x.  [assumption].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
19 A

Next we show a modal version of Conjunctive Sufficiency: $\Box(\varphi\wedge\psi) \vdash \Box(\varphi\to\psi)$.

In [26]:
goal = read_expr('Box(And(x,y)) = And(Box(And(x,y)),Box(If(x,y)))')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27507 was started by halcrow on iMac.local,
Wed Feb  9 20:17:24 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.19 (+ 0.01) seconds.
% Length of proof is 59.
% Level of proof is 9.
% Maximum clause weight is 27.000.
% Given clauses 326.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
11 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
12 Box(And(x,y)) = And(Box(And(x,y)),Box(If(x,y))).  [goal].
14 And(x,x) = x.  [assumption].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
19 And

Next we show that if $\varphi\vdash\psi$, then $\vdash \varphi\to\Box\psi$ (Must Intro).

In [27]:
must_intro = read_expr('x = And(x,y) -> If(x,Box(y)) = Top')
prover = Prover9Command(must_intro, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27509 was started by halcrow on iMac.local,
Wed Feb  9 20:17:24 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.03 (+ 0.00) seconds.
% Length of proof is 42.
% Level of proof is 6.
% Maximum clause weight is 20.000.
% Given clauses 153.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
12 x = And(x,y) -> If(x,Box(y)) = Top.  [goal].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
21 Or(x,And(x,y)) = x.  [assumption].
22 And(x,Or(x,y)) = x.  [assumption].
24 Or(x,Top) = Top.  [assumption].
25 And(x,Not(

Next we show that if $\varphi\vdash\psi$, then $\psi\to\bot\vdash \varphi\to\bot$ (Absurdity Reversal).

In [28]:
goal = read_expr('x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot)) ')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27511 was started by halcrow on iMac.local,
Wed Feb  9 20:17:24 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.08 (+ 0.00) seconds.
% Length of proof is 46.
% Level of proof is 7.
% Maximum clause weight is 20.000.
% Given clauses 263.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
12 x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot)).  [goal].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
19 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
20 And(And(x,y),z) = And(x,And(y,z)).  [copy(19),flip

Next we show $(\varphi\to\bot)\vdash \neg\varphi$ (Ad Falsum).

In [29]:
goal = read_expr('If(x,Bot) = And(If(x,Bot),Not(x))')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27513 was started by halcrow on iMac.local,
Wed Feb  9 20:17:24 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 22.
% Level of proof is 4.
% Maximum clause weight is 18.000.
% Given clauses 30.


7 B(x) -> And(If(y,x),Not(x)) = And(And(If(y,x),Not(x)),Not(y)).  [assumption].
12 If(x,Bot) = And(If(x,Bot),Not(x)).  [goal].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
22 And(x,Or(x,y)) = x.  [assumption].
24 Or(x,Top) = Top.  [assumption].
25 And(x,Not(x)) = Bot.  [assumption].
26 Or(x,Not(x)) = Top.  [assumption

Next we show $\varphi\vdash \top\to \varphi$ and $\top\to \varphi \vdash\varphi$.

In [30]:
goal = read_expr('x = And(x,If(Top,x))')
prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 27515 was started by halcrow on iMac.local,
Wed Feb  9 20:17:24 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.09 (+ 0.01) seconds.
% Length of proof is 51.
% Level of proof is 8.
% Maximum clause weight is 24.000.
% Given clauses 265.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
11 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
12 x = And(x,If(Top,x)).  [goal].
15 Or(x,y) = Or(y,x).  [assumption].
16 And(x,y) = And(y,x).  [assumption].
19 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
20 And(And(x

## 10. Avoiding collapse<a id='10'></a>

Next we show that $\alpha\to\beta \not\equiv \neg\alpha\vee\beta $ (No Simple Collapse).

In [31]:
goal = read_expr('B(x) & B(y) -> If(x,y) = Or(Not(x),y)')
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

c2 = 0.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

  B(0).
  B(1).
  B(2).
  B(3).



## 11. Modalized Import-Export<a id='11'></a>

Consider, $(\varphi\to \Diamond(\varphi\wedge\psi))\wedge (\varphi \to (\psi\to\chi)) \equiv (\varphi\to \Diamond(\varphi\wedge\psi)) \wedge ((\varphi\wedge\psi)\to\chi)$ (Modalized Import-Export), which is proved in the paper. Unfortunately, Prover9 does not find the proof before timing out.

In [32]:
mie = read_expr('And(If(x,Diamond(And(x,y))),If(x,If(y,z))) = And(If(x,Diamond(And(x,y))),If(And(x,y),z))')

In [33]:
#A simple version of mie with x, y, and z non-epistemic
simp_mie = read_expr('((B(x) & B(z)) & B(z)) -> And(If(x,Diamond(And(x,y))),If(x,If(y,z))) = And(If(x,Diamond(And(x,y))),If(And(x,y),z))')

In [34]:
#goal = mie
#prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice, timeout=10000)
#prover.prove()
#print(prover.proof())

In [35]:
#goal = mie
#mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
#mb.build_model()
#print(mb.model(format='cooked'))

Mace4 is able to show that without Scopelessness and Must Preservation, Modalized Import-Export is not derivable.

In [36]:
goal = mie
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not (axiom == scopeless or axiom == must_preserve)])

# Scopelessness and Must Preservation are not included

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 1

% Interpretation of size 8

Bot = 0.

Top = 1.

c1 = 2.

c2 = 6.

c3 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 5.
Box(3) = 3.
Box(4) = 3.
Box(5) = 5.
Box(6) = 5.
Box(7) = 7.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 4.
Diamond(4) = 4.
Diamond(5) = 2.
Diamond(6) = 6.
Diamond(7) = 4.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 5.
And(2,6) = 2.
And(2,7) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 3.
And(3,5) = 0.
And(3,6) = 0.
And(3,7) = 7.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 3.
And(4,4) = 4.
And(4,5) = 0.
And(4,6) = 0.
And(4,7) = 7.
And(5,0) = 0.


In [37]:
#The non-epistemic version of qie is also not derivable without Scopelessness and Must Preservation
goal = simp_mie
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not (axiom == scopeless or axiom == must_preserve)])

# Scopelessness and Must Preservation are not included

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 90

% Interpretation of size 12

Bot = 0.

Top = 1.

c1 = 2.

c2 = 2.

c3 = 4.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 8.
Box(6) = 6.
Box(7) = 9.
Box(8) = 8.
Box(9) = 9.
Box(10) = 4.
Box(11) = 6.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 10.
Diamond(5) = 5.
Diamond(6) = 11.
Diamond(7) = 7.
Diamond(8) = 5.
Diamond(9) = 7.
Diamond(10) = 10.
Diamond(11) = 11.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 10.
Not(9) = 11.
Not(10) = 8.
Not(11) = 9.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(0,10) = 0.
And(0,11) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(1,10) = 10.
And(1,11) = 11.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
An

## 12. Qualified Collapse<a id='12'></a>

In Proposition 6.20 in the paper, we consider Qualified Collapse: $\psi\wedge (\psi\to \Diamond (\varphi\wedge\psi))\vdash \varphi\to\psi$.

In [38]:
q_collapse = read_expr('And(y,If(y,Diamond(And(x,y)))) = And(And(y,If(y,Diamond(And(x,y)))), If(x,y))')

In [39]:
#A simple version of q_collapse with x,y non-epistemic
simp_q_collapse = read_expr('(B(x) & B(y)) -> And(y,If(y,Diamond(And(x,y)))) = And(And(y,If(y,Diamond(And(x,y)))), If(x,y))')

In [40]:
#If we drop distributivity from the assumptions of Proposition 6.20, 
#then Mace4 finds a counterexample to even simple q_collapse,
#which satisfies not only modal_ortho_boolean_lattice but even epistemic_ortho_boolean_lattice

goal = simp_q_collapse
mb = MaceCommand(goal, assumptions = epistemic_ortho_boolean_lattice + [if_and,if_top,ident,stm,mie])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 1.

c2 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.
Box(4) = 4.
Box(5) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.
Diamond(4) = 2.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 4.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 3.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 4.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 3.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 4.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 3.
If

In [41]:
#In fact, Mace4 finds a model falsifying q_collapse while satisfying 
#all of cond_epistemic_ortho_boolean_lattice.

goal = q_collapse
mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 8

Bot = 0.

Top = 1.

c1 = 2.

c2 = 6.

Box(0) = 0.
Box(1) = 1.
Box(2) = 5.
Box(3) = 3.
Box(4) = 4.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 4.
Diamond(4) = 4.
Diamond(5) = 5.
Diamond(6) = 6.
Diamond(7) = 7.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 5.
And(2,6) = 5.
And(2,7) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 3.
And(3,5) = 0.
And(3,6) = 0.
And(3,7) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 3.
And(4,4) = 4.
And(4,5) = 0.
And(4,6) = 0.
And(4,7) = 7.
And(5,0) = 0.
And(5,1) 

In [42]:
#It's also noteworthy that the assumptions of Proposition 6.20 
#do not entail our episteme axiom

goal = episteme
mb = MaceCommand(goal, assumptions = modal_ortho_boolean_lattice + [if_and,if_top,ident,stm,mie])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 0.
Box(3) = 1.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 0.
Diamond(3) = 1.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 0.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

  B(0).
  B(1).
- B(2).
- B(3).



Unfortunately, to date Mace4 has not found a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice, while Prover9 has not found a proof of simp_q_collapse from cond_epistemic_ortho_boolean_lattice.

In [43]:
#goal = simp_q_collapse
#mb = MaceCommand(goal, assumptions = cond_epistemic_ortho_boolean_lattice)
#mb.build_model()
#print(mb.model(format='cooked'))

In [44]:
#goal = simp_q_collapse
#prover = Prover9Command(goal, assumptions = cond_epistemic_ortho_boolean_lattice, timeout=1000000)
#prover.prove()
#print(prover.proof())

However, we have some partial results.

First, Mace4 can find a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice except for simp_cs and simp_ni.

In [45]:
goal = simp_q_collapse
mb = MaceCommand(goal, assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                      if not (axiom == simp_cs or axiom == simp_ni)])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 1.

c2 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 4.
Box(3) = 3.
Box(4) = 4.
Box(5) = 5.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 5.
Diamond(4) = 4.
Diamond(5) = 5.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 4.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 3.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 4.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 3.
And(5,4) = 0.
And(5,5) = 5.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(0,4) = 1.
If(0,5) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 4.
If(1,3) = 3.
If(1,4) = 4.
If(1,5) = 5.
If

Second, Mace4 can find a model falsifying simp_q_collapse while satisfying all of cond_epistemic_ortho_boolean_lattice except for flat (although a restricted version of flat works), scopeless, and must_preserve.

In [46]:
simp_flat = read_expr('B(x) -> If(x,If(And(x,y),z)) = If(And(x,y),z)')

goal = simp_q_collapse
mb = MaceCommand(goal, 
                 assumptions = [axiom for axiom in cond_epistemic_ortho_boolean_lattice 
                                if not (axiom == flat or axiom == scopeless or axiom == must_preserve)] 
                                + [simp_flat])

mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 88

% Interpretation of size 12

Bot = 0.

Top = 1.

c1 = 2.

c2 = 6.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 5.
Box(4) = 2.
Box(5) = 5.
Box(6) = 6.
Box(7) = 7.
Box(8) = 8.
Box(9) = 11.
Box(10) = 8.
Box(11) = 11.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 3.
Diamond(6) = 6.
Diamond(7) = 7.
Diamond(8) = 10.
Diamond(9) = 9.
Diamond(10) = 10.
Diamond(11) = 9.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.
Not(6) = 7.
Not(7) = 6.
Not(8) = 9.
Not(9) = 8.
Not(10) = 11.
Not(11) = 10.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(0,6) = 0.
And(0,7) = 0.
And(0,8) = 0.
And(0,9) = 0.
And(0,10) = 0.
And(0,11) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(1,6) = 6.
And(1,7) = 7.
And(1,8) = 8.
And(1,9) = 9.
And(1,10) = 10.
And(1,11) = 11.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 

## 13. Provable principles for which Prover9 does not find a proof <a id='12'></a>

Consider $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$ (Conditional Modus Tollens), which is provable as follows:

1. $\psi\to\bot \vdash \psi\to\varphi$ and $\psi\to\bot\vdash\psi\to\bot$, so $\psi\to\bot \vdash (\psi\to\varphi)\wedge (\psi\to\bot)$
2. By Simple Cautious Monotonicity, $(\psi\to\varphi)\wedge (\psi\to\bot)\vdash (\psi\wedge\varphi)\to\bot\vdash (\varphi\wedge\psi)\to\bot$.
3. By 1 and 2, $\psi\to\bot \vdash (\varphi\wedge\psi)\to\bot$.
4. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge (\varphi\wedge\psi)\to\bot\vdash \varphi\to\bot$.
5. By 3 and 4, $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$.

Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [47]:
#goal = read_expr('And(If(x,y),If(y,Bot)) = And(And(If(x,y),If(y,Bot)),If(x,Bot))')
#prover = Prover9Command(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#prover.prove()
#print(prover.proof())

In [48]:
#goal = read_expr('And(If(x,y),If(y,Bot)) = And(And(If(x,y),If(y,Bot)),If(x,Bot))')
#mb = MaceCommand(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#mb.build_model()
#print(mb.model(format='cooked'))

Consider, for non-epistemic $\beta$, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$ (Simple Reciprocity), which is provable as follows:

1. By Simple Cautious Monotonicity,  $(\psi\to\varphi)\wedge (\psi\to\beta)\vdash (\psi\wedge\varphi)\to\beta\vdash (\varphi\wedge\psi)\to\beta$.

2. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge ((\varphi\wedge\psi)\to\beta)\vdash \varphi\to\beta$.

3. By 1 and 2, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$.

Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [49]:
#goal = read_expr('B(z) -> And(And(If(x,y),If(y,x)),If(y,z)) = And(And(And(If(x,y),If(y,x)),If(y,z)),If(x,z))')
#prover = Prover9Command(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#prover.prove()
#print(prover.proof())

In [50]:
#goal = read_expr('B(z) -> And(And(If(x,y),If(y,x)),If(y,z)) = And(And(And(If(x,y),If(y,x)),If(y,z)),If(x,z))')
#mb = MaceCommand(goal, assumptions = ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,simp_caut_mon])
#mb.build_model()
#print(mb.model(format='cooked'))